# NBA Game Prediction - Data Wrangling (Season 2022+) - L3 Rolling Window

This notebook prepares the data for modeling NBA game outcomes using **3-game rolling windows**.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)

print("Packages loaded successfully!")

Packages loaded successfully!


## 1. Load Data

In [2]:
games = pd.read_csv('team_traditional.csv')
games['date'] = pd.to_datetime(games['date'])
games = games[games['season'] >= 2022].copy()

print(f"Loaded {len(games):,} games (season 2022+)")
print(f"Date range: {games['date'].min().date()} to {games['date'].max().date()}")
print(f"Teams: {games['team'].nunique()}")
print(f"Seasons: {sorted(games['season'].unique())}")

Loaded 10,562 games (season 2022+)
Date range: 2021-10-19 to 2025-06-22
Teams: 30
Seasons: [np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]


## 2. Calculate Opponent Stats

In [3]:
# Opponent points
opp_pts = games.groupby('gameid')['PTS'].transform(lambda x: x.iloc[::-1].values)
games['opp_pts'] = opp_pts

# Possessions (standard NBA formula)
games['possessions'] = games['FGA'] + 0.44 * games['FTA'] - games['OREB'] + games['TOV']

# Opponent possessions
opp_poss = games.groupby('gameid')['possessions'].transform(lambda x: x.iloc[::-1].values)
games['opp_possessions'] = opp_poss

# Opponent FGA, 3PA, 2PA (for block %)
opp_fga = games.groupby('gameid')['FGA'].transform(lambda x: x.iloc[::-1].values)
games['opp_fga'] = opp_fga
opp_3pa = games.groupby('gameid')['3PA'].transform(lambda x: x.iloc[::-1].values)
games['opp_3pa'] = opp_3pa
games['opp_2pa'] = games['opp_fga'] - games['opp_3pa']

# Opponent DREB (for OREB %)
opp_dreb = games.groupby('gameid')['DREB'].transform(lambda x: x.iloc[::-1].values)
games['opp_dreb'] = opp_dreb

print("✓ Opponent stats calculated")

✓ Opponent stats calculated


## 3. Calculate Advanced Metrics

In [4]:
# Offensive Rating: Points per 100 possessions
games['off_rtg'] = (games['PTS'] / games['possessions']) * 100

# Defensive Rating: Points allowed per 100 possessions
games['def_rtg'] = (games['opp_pts'] / games['opp_possessions']) * 100

# Net Rating
games['net_rtg'] = games['off_rtg'] - games['def_rtg']

# Pace
games['pace'] = (games['possessions'] + games['opp_possessions']) / 2

# Turnover %
games['to_pct'] = (games['TOV'] / games['possessions']) * 100

# Free Throw Rate
games['ft_rate'] = games['FTA'] / games['FGA']

# Offensive Rebound %
games['oreb_pct'] = games['OREB'] / (games['OREB'] + games['opp_dreb']) * 100

# Assist Ratio
games['ast_ratio'] = (games['AST'] / games['possessions']) * 100

# Steal %
games['stl_pct'] = (games['STL'] / games['opp_possessions']) * 100

# Block %
games['blk_pct'] = (games['BLK'] / games['opp_2pa']) * 100

# Effective FG%
games['eFG%'] = (games['FGM'] + 0.5 * games['3PM']) / games['FGA']

# 3PA Rate
games['3pa_rate'] = games['3PA'] / games['FGA']

print("✓ Advanced metrics calculated")

✓ Advanced metrics calculated


## 4. Calculate Rolling Features (L3 - Last 3 Games)

**Critical**: We use `.shift(1)` to ensure we only use games BEFORE the current one (no data leakage).

In [5]:
# Sort chronologically per team
games = games.sort_values(['team', 'season', 'date']).reset_index(drop=True)

def rolling_avg(df, column, window=3, stat='mean'):
    """Calculate rolling average for a column, excluding current game"""
    grouped = df.groupby(['team', 'season'])[column].rolling(window, min_periods=1 if stat == 'mean' else 2)
    
    if stat == 'mean':
        result = grouped.mean()
    elif stat == 'std':
        result = grouped.std()
    result = result.groupby(level=['team', 'season']).shift(1)
    return result.reset_index(level=['team', 'season'], drop=True)

In [6]:
# L3 WINDOW
window = 3

# Advanced Efficiency Metrics
games['off_rtg_L3'] = rolling_avg(games, 'off_rtg', window)
games['def_rtg_L3'] = rolling_avg(games, 'def_rtg', window)
games['net_rtg_L3'] = rolling_avg(games, 'net_rtg', window)

# Shooting Efficiency
games['efg_pct_L3'] = rolling_avg(games, 'eFG%', window)
games['3p_pct_L3'] = rolling_avg(games, '3P%', window)
games['3pa_rate_L3'] = rolling_avg(games, '3pa_rate', window)

# Form/Momentum
games['win_pct_L3'] = rolling_avg(games, 'win', window)

# Tempo & Ball Control
games['pace_L3'] = rolling_avg(games, 'pace', window)
games['to_pct_L3'] = rolling_avg(games, 'to_pct', window)
games['ft_rate_L3'] = rolling_avg(games, 'ft_rate', window)

# Rebounding & Playmaking
games['oreb_pct_L3'] = rolling_avg(games, 'oreb_pct', window)
games['ast_ratio_L3'] = rolling_avg(games, 'ast_ratio', window)

# Defensive Stats
games['stl_pct_L3'] = rolling_avg(games, 'stl_pct', window)
games['blk_pct_L3'] = rolling_avg(games, 'blk_pct', window)

# Consistency (point volatility)
games['pts_std_L3'] = rolling_avg(games, 'PTS', window, stat='std')

print(f"✓ L3 rolling features calculated (window={window})")

✓ L3 rolling features calculated (window=3)


## 5. Calculate Contextual Features

In [7]:
# Rest days
games['rest_days'] = games.groupby(['team', 'season'])['date'].diff().dt.days

# Back-to-back
games['is_b2b'] = (games['rest_days'] <= 1).astype(int)

# Game number
games['game_num'] = games.groupby(['team', 'season']).cumcount() + 1

# Win streak
def calculate_win_streak(series):
    streaks = []
    for i in range(len(series)):
        if i == 0:
            streaks.append(0)
        else:
            recent_wins = series.iloc[:i]
            if len(recent_wins) == 0:
                streaks.append(0)
            else:
                current = recent_wins.iloc[-1]
                streak = 0
                for j in range(len(recent_wins)-1, -1, -1):
                    if recent_wins.iloc[j] == current:
                        streak += 1
                    else:
                        break
                streaks.append(streak if current == 1 else -streak)
    return streaks

games['win_streak'] = games.groupby(['team', 'season'])['win'].transform(calculate_win_streak)

print("✓ Contextual features calculated")

✓ Contextual features calculated


## 6. Create Matchup Dataset

In [8]:
# Home team data
home = games[games['team'] == games['home']].copy()
home.columns = [f'{col}_home' if col not in ['gameid', 'date', 'season'] else col 
                for col in home.columns]

# Away team data
away = games[games['team'] == games['away']].copy()
away.columns = [f'{col}_away' if col not in ['gameid', 'date', 'season'] else col
                for col in away.columns]

# Merge
matchups = home.merge(away, on=['gameid', 'date', 'season'], how='inner')

print(f"✓ Created {len(matchups):,} matchups")

✓ Created 5,276 matchups


## 7. Calculate Differentials (away - home)

In [9]:
# Advanced Efficiency Gaps
matchups['off_rtg_L3_diff'] = matchups['off_rtg_L3_away'] - matchups['off_rtg_L3_home']
matchups['def_rtg_L3_diff'] = matchups['def_rtg_L3_away'] - matchups['def_rtg_L3_home']
matchups['net_rtg_L3_diff'] = matchups['net_rtg_L3_away'] - matchups['net_rtg_L3_home']

# Shooting Efficiency Gaps
matchups['efg_pct_L3_diff'] = matchups['efg_pct_L3_away'] - matchups['efg_pct_L3_home']
matchups['3p_pct_L3_diff'] = matchups['3p_pct_L3_away'] - matchups['3p_pct_L3_home']
matchups['3pa_rate_L3_diff'] = matchups['3pa_rate_L3_away'] - matchups['3pa_rate_L3_home']

# Form Gap
matchups['win_pct_L3_diff'] = matchups['win_pct_L3_away'] - matchups['win_pct_L3_home']

# Tempo & Ball Control Gaps
matchups['pace_L3_diff'] = matchups['pace_L3_away'] - matchups['pace_L3_home']
matchups['to_pct_L3_diff'] = matchups['to_pct_L3_away'] - matchups['to_pct_L3_home']
matchups['ft_rate_L3_diff'] = matchups['ft_rate_L3_away'] - matchups['ft_rate_L3_home']

# Rebounding & Playmaking Gaps
matchups['oreb_pct_L3_diff'] = matchups['oreb_pct_L3_away'] - matchups['oreb_pct_L3_home']
matchups['ast_ratio_L3_diff'] = matchups['ast_ratio_L3_away'] - matchups['ast_ratio_L3_home']

# Defensive Gaps
matchups['stl_pct_L3_diff'] = matchups['stl_pct_L3_away'] - matchups['stl_pct_L3_home']
matchups['blk_pct_L3_diff'] = matchups['blk_pct_L3_away'] - matchups['blk_pct_L3_home']

# Consistency & Momentum Gaps
matchups['pts_std_L3_diff'] = matchups['pts_std_L3_away'] - matchups['pts_std_L3_home']
matchups['win_streak_diff'] = matchups['win_streak_away'] - matchups['win_streak_home']

# Rest Advantage
matchups['rest_advantage'] = matchups['rest_days_away'] - matchups['rest_days_home']

print("✓ Differentials calculated")

✓ Differentials calculated


## 8. Clean Data

In [10]:
print(f"Rows before cleaning: {len(matchups):,}")

matchups_clean = matchups.dropna(subset=[
    'off_rtg_L3_home', 'off_rtg_L3_away',
    'def_rtg_L3_home', 'def_rtg_L3_away',
    'win_pct_L3_home', 'win_pct_L3_away',
    'efg_pct_L3_home', 'efg_pct_L3_away', 
    'pts_std_L3_diff'
])

print(f"Rows after cleaning: {len(matchups_clean):,}")
print(f"Removed: {len(matchups) - len(matchups_clean):,} rows")

Rows before cleaning: 5,276
Rows after cleaning: 5,145
Removed: 131 rows


## 9. Save Data

In [11]:
# Save game data
games_clean = games.dropna(subset=['off_rtg_L3'])
games_clean.to_csv('nba_games_with_features_L3.csv', index=False)
print(f"✓ Saved: nba_games_with_features_L3.csv ({len(games_clean):,} rows)")

# Save matchup data
matchups_clean.to_csv('nba_matchups_with_features_L3.csv', index=False)
print(f"✓ Saved: nba_matchups_with_features_L3.csv ({len(matchups_clean):,} rows)")

print("\n✓ DATA WRANGLING COMPLETE! (L3 Rolling Window)")

✓ Saved: nba_games_with_features_L3.csv (10,442 rows)
✓ Saved: nba_matchups_with_features_L3.csv (5,145 rows)

✓ DATA WRANGLING COMPLETE! (L3 Rolling Window)
